In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import geopandas

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
dataf_mddr1116 = pd.read_csv('dbs/travel_survey_se_deso/se_rvu_distribution_mdre1116_deso.csv', dtype = {'d_a_kl' : 'Int32', 'd_b_kl' : 'Int32', 'd_a_pkt' : 'Int32', 'd_a_sve': 'Int32', 'd_b_sve': 'Int32','h_a_lan': 'Int32','h_b_lan': 'Int32'}, delimiter=',',  encoding='latin-1')


print('The number of people= ', len(dataf_mddr1116.uenr.unique()))

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (181,370,379) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


The number of people=  38258


In [4]:
variables_dict = {"uenr":"sub_id", "bost_lan":"home_county", 'aplats':'employee',
                  'bost_komk':'home_municipality',
                   "arb_komk":'workplace_municipality',
                  "arb_fard":"commute_mode","arb_km":"commute_distance", "arb_lan":"workplace_county","arb_tid":"commute_time",
                  "i_datum":"date", "uedag":"day_of_week", "h_nummer":"trip_id", "d_nummer":"trip_main_id", "r_nummer":"trip_seg_id",
                  "d_ford":"trip_main_mode", "d_km":"distance_main", "d_restid":"tt_main", "d_areall":"trip_purpose_main",
                  "d_a_sve":"origin_main_domestic","d_b_sve":"desti_main_domestic",
                  'd_a_kl':"origin_main_time", 'd_b_kl': "desti_main_time",
                  "r_ford":"trip_seg_mode", "r_km":"distance_seg", # , "r_restid":"tt_seg"
                  "h_a_kl":"origin_time", "h_a_lan":"origin_county", "h_b_kl":"desti_time",
                  "h_b_lan":"desti_county", "h_km":"distance", "h_restid":"tt",
                  "h_ford":"mode", "h_are":"trip_purpose",
                  "vikt":"trip_weight", "vikt_k":"sub_weight",
                  'd_a_deso':"origin_main_deso", 'd_b_deso':"desti_main_deso",
                  'd_a_komk':"origin_main_municipality", 'd_b_komk':"desti_main_municipality",
                  'h_a_deso':"origin_deso", 'h_b_deso':"desti_deso",
                  'arb_deso':'workplace_deso', 'bost_deso':'home_deso', 'd_a_pkt':'origin_act_typ','d_b_pkt':'desti_act_typ'}

In [5]:
dataf_mddr1116= dataf_mddr1116.rename(columns=variables_dict)
dataf_mddr1116 =  dataf_mddr1116.sort_values(['sub_id', 'trip_id','trip_main_id', 'trip_seg_id'])

In [6]:
#select individuals based on days of a week (consider only weekdays)

dataf_mddr1116 = dataf_mddr1116[dataf_mddr1116.day_of_week.isin([1,2,3,4,5])]

# drop people less than 18 years old, keep only adults

dataf_mddr1116= dataf_mddr1116[dataf_mddr1116.age>17]

print('The number of people having a weekday activities= ', len(dataf_mddr1116.sub_id.unique()))

The number of people having a weekday activities=  24453


In [7]:
selected_columns = list(variables_dict.values())
selected_columns = ['sub_id', 'employee',
 'trip_id',
 'trip_main_id',
 'origin_main_time',
 'desti_main_time',
 'origin_act_typ',
 'desti_act_typ',
 'origin_main_domestic',
 'desti_main_domestic',
 'origin_main_deso',
 'desti_main_deso',
 'origin_main_municipality',
 'desti_main_municipality',
 'origin_county',
 'desti_county',
 'home_deso',
 'home_county',
 'home_municipality',
 'trip_weight',
 'sub_weight', "distance", "distance_main", "distance_seg","commute_distance"
 ]

In [8]:
dataf_mddr1116= dataf_mddr1116[selected_columns]

### calculate 99th percentile of the travel distances

In [9]:
dataf_mddr1116_per = dataf_mddr1116.copy()
dataf_mddr1116_per['distance_seg'] = dataf_mddr1116_per['distance_seg'].fillna(0)
null_main_dist = dataf_mddr1116_per[dataf_mddr1116_per.distance_main.isnull()].index

# sum the distances in segments of the main trip
dataf_mddr1116_seg = dataf_mddr1116_per[dataf_mddr1116_per.index.isin(null_main_dist)].groupby(['sub_id', 'trip_id', 'trip_main_id'])['distance_seg'].sum()
dataf_mddr1116_seg = dataf_mddr1116_seg.reset_index()

# drop trip legs, keep only main trips
dataf_mddr1116_per = dataf_mddr1116_per.drop_duplicates(['sub_id', 'trip_id', 'trip_main_id'])

# write the sum of segment distances to the main trip distance if it is null
dataf_mddr1116_per['distance_main'][dataf_mddr1116_per.index.isin(null_main_dist)] = dataf_mddr1116_seg['distance_seg'].values


dataf_mddr1116_per = dataf_mddr1116_per[dataf_mddr1116_per.distance_main > 0]
dataf_mddr1116_per.distance_main.quantile(0.99)

204.51199999999955

### calculate avg distance between home and all activity types
### calculate commuting distance

In [10]:
# drop trip legs, keep only main trips

dataf_mddr1116 = dataf_mddr1116.drop_duplicates(['sub_id','trip_id','trip_main_id'])

In [11]:
# adjust the activity types

dataf_mddr1116['origin_act_typ'].replace([2, 3, 9], 1, inplace=True)
dataf_mddr1116['desti_act_typ'].replace([2, 3, 9], 1, inplace=True)


dataf_mddr1116['origin_act_typ'].replace([5], 4, inplace=True)
dataf_mddr1116['desti_act_typ'].replace([5], 4, inplace=True)

In [12]:
# copy deso information from home deso column, if main deso is null

dataf_mddr1116['desti_main_deso'][dataf_mddr1116.desti_main_deso.isnull()] = np.where(dataf_mddr1116['desti_main_deso'][dataf_mddr1116.desti_main_deso.isnull()] , dataf_mddr1116['home_deso'][dataf_mddr1116.desti_main_deso.isnull()], dataf_mddr1116['desti_main_deso'][dataf_mddr1116.desti_main_deso.isnull()])

In [13]:
# concat the home information from destination and origin points

dataf_ppl = dataf_mddr1116[['sub_id','desti_main_deso', 'desti_act_typ']][dataf_mddr1116.desti_act_typ == 1]

dataf_ppl = dataf_ppl.rename(columns={'desti_main_deso':'origin_main_deso', 'desti_act_typ':'origin_act_typ'})

In [14]:
dataf_ppl = pd.concat([dataf_ppl,dataf_mddr1116[['sub_id','origin_main_deso', 'origin_act_typ']][dataf_mddr1116.origin_act_typ == 1]])


dataf_ppl = dataf_ppl.drop_duplicates(subset= ['sub_id'])

In [15]:
# drop individuals if the home deso column is null

ppl_no_residence = dataf_ppl['sub_id'][dataf_ppl.origin_main_deso.isnull()].unique()

dataf_ppl = dataf_ppl[~(dataf_ppl.sub_id.isin(ppl_no_residence))]

In [16]:
# concat activity information from destination and origin points


dataf_ppl_dest = dataf_mddr1116[['sub_id','origin_main_deso', 'origin_act_typ']][dataf_mddr1116.origin_act_typ != 1]


dataf_ppl_dest = dataf_ppl_dest.rename(columns={'origin_main_deso':'desti_main_deso', 'origin_act_typ':'desti_act_typ'})

In [17]:
dataf_ppl_dest = pd.concat([dataf_ppl_dest,dataf_mddr1116[['sub_id','desti_main_deso', 'desti_act_typ']][dataf_mddr1116.desti_act_typ != 1]])

dataf_ppl_dest = dataf_ppl_dest.drop_duplicates(subset= ['sub_id','desti_act_typ', 'desti_main_deso'])

In [18]:
# merge home and all activity type tables by id

dataf_ppl = pd.merge(dataf_ppl, dataf_ppl_dest, on='sub_id', how='left')

dataf_ppl = dataf_ppl[['sub_id', 'origin_act_typ', 'desti_act_typ', 'origin_main_deso', 'desti_main_deso']]

In [19]:
# drop columns if deso and activity type is null

dataf_ppl= dataf_ppl.dropna(subset=["desti_act_typ", 'desti_main_deso'])

In [20]:
len(dataf_ppl['sub_id'].unique())

17633

In [21]:
# read the deso data with distances between the zones


df_dist_DeSO= pd.read_pickle(r'.\dbs\intermediate\df_DeSO_dist.pkl')

In [22]:
# bring distance information to the activity table

dataf_ppl = pd.merge(dataf_ppl, df_dist_DeSO, left_on=['origin_main_deso', 'desti_main_deso'], right_on=['origin_deso', 'desti_deso'], how='left')

In [23]:
#calculate avg distance between home and all activity types in an individual schedule

dataf_ppl_dist = dataf_ppl.groupby('sub_id')['distance'].median().reset_index().rename(columns={'distance':'avg_dist'})

In [24]:
dataf_ppl = pd.merge(dataf_ppl, dataf_ppl_dist, on=['sub_id'], how='left')

In [25]:
#calculate the commute distance

dataf_ppl_commute = dataf_ppl[(dataf_ppl.origin_act_typ==1)&(dataf_ppl.desti_act_typ==4)]
dataf_ppl_commute = dataf_ppl_commute[['sub_id', 'distance']].rename(columns={'distance':'com_distance'})
dataf_ppl_commute['commute']=1

In [26]:
dataf_ppl = pd.merge(dataf_ppl, dataf_ppl_commute, on=['sub_id'], how='left')

dataf_ppl['commute'].fillna(0, inplace=True)

dataf_ppl['commute']= dataf_ppl['commute'].astype(int)

In [27]:
# add employee column

dataf_ppl = pd.merge(dataf_ppl, dataf_mddr1116[['sub_id', 'employee']].drop_duplicates(subset='sub_id'), on=['sub_id'], how='left')

In [28]:
dataf_ppl = pd.merge(dataf_ppl, dataf_mddr1116[['sub_id','commute_distance']].drop_duplicates(subset='sub_id'), on=['sub_id'], how='left')

In [29]:
dataf_ppl['commute_distance']=dataf_ppl['commute_distance']*1000

In [30]:
dataf_ppl['com_distance'][(dataf_ppl.employee==1)&(dataf_ppl.com_distance.isnull())] = dataf_ppl['commute_distance'][(dataf_ppl.employee==1)&(dataf_ppl.com_distance.isnull())]


C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
dataf_ppl.drop_duplicates(subset=['sub_id'], inplace=True)


In [32]:
len(dataf_ppl[(dataf_ppl.employee==1)&(dataf_ppl.com_distance.notnull())])

11462

In [33]:
len(dataf_ppl[(dataf_ppl.employee==1)&(dataf_ppl.com_distance.isnull())])


792

In [34]:
dataf_ppl['com_distance'][(dataf_ppl.employee==1)] = dataf_ppl['com_distance'][(dataf_ppl.employee==1)].fillna(dataf_ppl['com_distance'][(dataf_ppl.employee==1)].median())


C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
dataf_ppl= dataf_ppl[['sub_id', 'origin_main_deso', 'avg_dist','employee', 'commute', 'com_distance']]

In [36]:
dataf_ppl.to_pickle(r'.\dbs\intermediate\df_survey_dist.pkl')
